# Hamiltonian Simulation

The hamiltonian that we simulate is $$H=(a Z_0+b Z_1+c Z_2 +d Z_3)^2$$

In [2]:
from pyqpanda import *
from pyqpanda.utils import *
from pyqpanda.Hamiltonian import PauliOperator, create_x_driver_hamiltonian
from pyqpanda.Algorithm.hamiltonian_simulation import *

from numpy import *
from pylab import *
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from copy import deepcopy

## An instance

In [3]:
Hp={'Z0':1,'Z1':2,'Z2':3,'Z3':4,'Z4':5,'Z5':7}
Hp=PauliOperator(Hp)
Hp=Hp*Hp

#fetch number of qubit from hamiltonian
qn=Hp.get_qubit_count()

#create the corresponding Sigma_X driver hamiltonian
Hd=create_x_driver_hamiltonian(qn)

target_result=['111010','000101','101001','010110']
step=200
slices=16
t=100
shots=100

result=adiabatic_simulation_with_configuration(qn_=qn, Hp_=Hp,Hd_=Hd,step_=step,slices_=slices,t_=t,shots_=shots)
print(result)
print(get_fidelity(result,shots,target_result))


TypeError: RZ(): incompatible function arguments. The following argument types are supported:
    1. (arg0: Qubit, arg1: float) -> QGate

Invoked with: <pyqpanda.pyQPanda.Qubit object at 0x00000260645E0F48>, (-0.03125+0j)

In [ ]:
# configure scanning params
step_start=1
step_end=30
step_step=8

step_linspace=range(step_start,step_end,step_step)

slice_start=1
slice_end=30
slice_step=8

slice_linspace=range(slice_start,slice_end,slice_step)

t_start=0.5
t_end=2.0
t_count=6

t_linspace=linspace(t_start,t_end,t_count)

meshgrid_slice, meshgrid_step=meshgrid(slice_linspace,step_linspace)

# number of shots
shots=1000

In [4]:
fidelity_list=list()

for i_t,t in enumerate(t_linspace):
    meshgrid_fidelity=deepcopy(meshgrid_slice)
    for i_step,step in enumerate(step_linspace):
        for i_slices,slices in enumerate(slice_linspace):
            result=simulate_with_configuration(Hp_=Hp, Hd_=Hd, step_=step,
                                        slices_=slices, t_=t, shots_=shots )
            fidelity=get_fidelity(result,shots)
            print('t=',t ,'step=',step,'slice=',slices,'Fidelity=',fidelity)
            meshgrid_fidelity[i_slices,i_step]=fidelity
            
    #save the result
    fidelity_list.append(meshgrid_fidelity)

NameError: name 't_linspace' is not defined

In [ ]:
def simulate_with_configuration_str(Hp_, Hd_, step_, slices_, t_ ):
    init()
    prog=QProg()
    q=qAlloc_many(6)
    c=cAlloc_many(6)
    prog.insert(single_gate_apply_to_all(gate=X, qubit_list=q))
    prog.insert(single_gate_apply_to_all(gate=H, qubit_list=q))

    for i in range(step_+1):
        Ht=Hp_.mul(i/step_)+Hd_.mul((step_-i)/step_)
        Ht.arrange()
        prog.insert(simulate_hamiltonian(q,Ht,t=t_,slices=slices_))

    prog.insert(meas_all(q,c))
    string=to_qrunes(prog)
    finalize()
    return string

prog_str=simulate_with_configuration_str(Hp,Hd,20,20,1)

67521


In [ ]:
s=prog_str.split('\n')

In [ ]:
print(len(s))

In [ ]:
import time

start =time.clock()
step=100
slices=100

prog_str=simulate_with_configuration_str(Hp,Hd,step,slices,1)
s=prog_str.split('\n')
print('code length:',len(s))


result=result=simulate_with_configuration(Hp_=Hp, Hd_=Hd, step_=step,
                                        slices_=slices, t_=10, shots_=shots )

fidelity=get_fidelity(result,shots)
print(fidelity)

end = time.clock()
print('Elasped time: %s Seconds'%(end-start))